**If you use our code, please cite:**

@misc{2024<br>
  title = {LLM Reference Architecture using Redis & Google Cloud Platform},<br>
  author = {Hamza Farooq, Darshil Modi, Kanwal Mehreen, Nazila Shafiei},<br>
  keywords = {Semantic Cache},<br>
  year = {2024},<br>
  copyright = {APACHE 2.0 license}<br>
}



# LLM Reference Architecture using Redis & Google Cloud Platform

<a href="https://colab.research.google.com/github/RedisVentures/redis-google-llms/blob/main/BigQuery_Palm_Redis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook serves as a getting started guide for working with LLMs on Google Cloud Platform with Redis Enterprise.

## Intro
Google's Vertex AI has expanded its capabilities by introducing [Generative AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview). This advanced technology comes with a specialized [in-console studio experience](https://cloud.google.com/vertex-ai/docs/generative-ai/start/quickstarts/quickstart), a [dedicated API](https://cloud.google.com/vertex-ai/docs/generative-ai/start/quickstarts/api-quickstart) and [Python SDK](https://cloud.google.com/vertex-ai/docs/python-sdk/use-vertex-ai-python-sdk) designed for deploying and managing instances of Google's powerful PaLM language models (more sample code). With a distinct focus on text generation, summarization, chat completion, and embedding creation, PaLM models are reshaping the boundaries of natural language processing and machine learning.

Redis Enterprise offers robust vector database features, with an efficient API for vector index creation, management, distance metric selection, similarity search, and hybrid filtering. When coupled with its versatile data structures - including lists, hashes, JSON, and sets - Redis Enterprise shines as the optimal solution for crafting high-quality Large Language Model (LLM)-based applications. It embodies a streamlined architecture and exceptional performance, making it an instrumental tool for production environments.

![](https://github.com/RedisVentures/redis-google-llms/blob/main/assets/GCP_RE_GenAI.drawio.png?raw=true)

Below we will work through several design patterns with Vertex AI LLMs and Redis Enterprise that will ensure optimal production performance.

___
## Contents
- Setup
    1. Prerequisites
    2. Create BigQuery Table
    3. Generate Embeddings
        
        a. Embed Text Data

    4. Load Embeddings to Redis
    5. Create Index
- Build LLM Applications
- LLM Design Patterns
    1. Semantic Search
    2. Retrieval Augmented Generation (RAG)
    3. Caching
    4. Memory
- Cleanup

___

# Setup

## 1. Prerequisites
Before we begin, we must install some required libraries, authenticate with Google, create a Redis database, and initialize other required components.

### Install required libraries

In [1]:
!pip install redis "google-cloud-aiplatform==1.25.0" --upgrade --user

In [2]:
!pip install huggingface datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
!pip uninstall -y shapely pygeos geopandas
# Install specific versions of shapely, pygeos, and geopandas known to be compatible
!pip install shapely==1.8.5.post1 pygeos==0.12.0 geopandas==0.10.2
# Upgrade google-cloud-aiplatform
!pip install -U google-cloud-aiplatform

Found existing installation: Shapely 1.8.5.post1
Uninstalling Shapely-1.8.5.post1:
  Successfully uninstalled Shapely-1.8.5.post1
Found existing installation: geopandas 1.0.1
Uninstalling geopandas-1.0.1:
  Successfully uninstalled geopandas-1.0.1
  Using cached Shapely-1.8.5.post1-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (43 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 3.4 MB/s eta 0:00:00
Using cached Shapely-1.8.5.post1-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (2.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 95.1 MB/s eta 0:00:00
  Attempting uninstall: shapely
    Found existing installation: shapely 2.0.6
    Uninstalling shapely-2.0.6:
      Successfully uninstalled shapely-2.0.6
ERROR: pip's dependency resolver does not currently take into account a

^^^ If prompted press the Restart button to restart the kernel. ^^^

### Install Redis locally (optional)
If you have a Redis db running elsewhere with [Redis Stack](https://redis.io/docs/about/about-stack/) installed, you don't need to run it on this machine. You can skip to the "Connect to Redis server" step.

In [4]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


### Using Free Redis Cloud account on GCP
You can also use Forever Free instance of Redis Cloud. To activate it:
- Head to https://redis.com/try-free/
- Register (using gmail-based registration is the easiest)
- Create New Subscription
- Use the following options:
    - Fixed plan, Gogle Cloud
    - New 30Mb Free database
- Create new RedisStack DB

If you are registering at Redis Cloud for the first time - the last few steps would be performed for you by default. Capture the host, port and default password of the new database. You can use these instead of default `localhost` based in the following code block.

### Connect to Redis server
Replace the connection params below with your own if you are connecting to an external Redis instance.

In [8]:
import os
import redis
from google.colab import userdata

# Redis connection params
REDIS_HOST = "localhost"
REDIS_PORT = "6379"
#REDIS_PASSWORD = "<REDIS PASSWORD>" ## using localhost -> has no password

# Create Redis client
redis_client = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT)
 # password=REDIS_PASSWORD)

# Test connection
redis_client.ping()
# redis_client.client_list()

True

In [9]:
# Clear Redis database (optional)
redis_client.flushdb()

True

### Authenticate to Google Cloud

In [14]:
from google.colab import auth
from google.colab import userdata

## get project and region from Colab secrets
PROJECT_ID = userdata.get('PROJECT_ID') #mavencourse3
REGION = userdata.get('REGION') #us-central1

auth.authenticate_user()
print('Authenticated. Project: "' + PROJECT_ID + '". Region: "' + REGION + '"')

Authenticated. Project: "maven-ai-advanced-course". Region: "europe-west4"


### Initialize Vertex AI Components



In [17]:
import vertexai

vertexai.init(project=PROJECT_ID, location=REGION)

In [ ]:
! gcloud services enable compute.googleapis.com aiplatform.googleapis.com storage.googleapis.com bigquery.googleapis.com --project "mavencourse3"

Operation "operations/acat.p2-124588469690-720428fc-9b28-4616-a134-0c825a194210" finished successfully.


## Test Vertex AI

In [18]:
!pip3 install google-cloud-aiplatform&gt==1.25


In [21]:
from vertexai.preview.language_models import TextEmbeddingModel
# model = TextEmbeddingModel.from_pretrained("textembedding-gecko")

# https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings/get-text-embeddings
text_embedding = "text-embedding-005"
model = TextEmbeddingModel.from_pretrained(text_embedding)


embeddings = model.get_embeddings(["Dinner in New York City"])
for embedding in embeddings:
  vector = embedding.values
  print(vector)

[-0.03962613642215729, 0.011793817393481731, 0.009970145300030708, -0.03202607110142708, 0.000994554371573031, -0.013233746401965618, 0.007667297963052988, 0.00974622555077076, 0.022329140454530716, 0.000425905454903841, -0.04578130692243576, 0.04914621636271477, 0.019248254597187042, -0.05525330454111099, -0.012394603341817856, 0.008795825764536858, 0.020506426692008972, -0.05978415161371231, -0.04084627702832222, -0.011847817339003086, -0.0241507887840271, -0.0590856596827507, -0.030450958758592606, 0.021381836384534836, -0.012182524427771568, -0.04211884364485741, -0.033370088785886765, 0.04521520063281059, 0.0029183567967265844, 0.021740682423114777, 0.055357418954372406, -0.030425427481532097, -0.010585686191916466, -0.014282512478530407, -0.017588140442967415, -0.03885775804519653, 0.04412943124771118, -0.013741955161094666, 0.012648004107177258, 0.0038484998513013124, 0.08307145535945892, 0.04639723151922226, -0.08787607401609421, 0.02600742131471634, 0.03631803020834923, -0.023

## 2. Create BigQuery Table
The second step involves preparing the dataset for our LLM applications. We utilize a free (public) hotel dataset from **hugging face**.

*Leveraging BigQuery is a common pattern for building ML applications because of it's powerful query and analytics capabilities.*

We will start by creating our own big query table for the dataset. Additionally, if you have a different dataset to work with you can follow a similar pattern, or even load a CSV from a Google Cloud Storage bucket into BigQuery.

### Create source table
First step is to create a new table from the public datasource.

In [22]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("traversaal-ai-hackathon/hotel_datasets")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

hotel_reviews_Istanbul.csv:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

hotel_reviews_San Francisco.csv:   0%|          | 0.00/1.53M [00:00<?, ?B/s]

hotel_reviews_london.csv:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

hotel_reviews_nyc.csv:   0%|          | 0.00/1.70M [00:00<?, ?B/s]

hotel_reviews_paris.csv:   0%|          | 0.00/1.76M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5997 [00:00<?, ? examples/s]

In [23]:
df=pd.DataFrame(dataset['train'])

In [24]:
df.head()

,hotel_name,hotel_description,review_title,review_text,rate,tripdate,hotel_url,hotel_image,price_range,rating_value,review_count,street_address,locality,country
0,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"An exceptional boutique hotel, great value for...",None,NaN,February 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
1,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,You can’t get better than this.,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
2,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Exceeds all expectations,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
3,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"Great Location, Fantastic Accommodations",None,NaN,August 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
4,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Perfection. It is all in the details.,None,NaN,June 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye


#Adding an index

In [25]:
df["id"] = df.index + 1

In [27]:
from google.cloud import bigquery

# Create bigquery client
bq = bigquery.Client(project=PROJECT_ID)

TABLE_NAME = "hotel_datasets"
DATASET_ID = f"{PROJECT_ID}.google_redis_llms"

# Create dataset
dataset = bigquery.Dataset(DATASET_ID)
dataset.location = "EU"
dataset = bq.create_dataset(dataset, timeout=30, exists_ok=True)

# Define table ID
TABLE_ID = f"{DATASET_ID}.{TABLE_NAME}"

In [28]:
# client = bigquery.Client()
job = bq.load_table_from_dataframe(
    df, TABLE_ID
)  # Make an API request.
job.result()

LoadJob<project=maven-ai-advanced-course, location=EU, id=a863962f-e2d2-4c9d-aa82-fb0d0c6cb3e7>

In [29]:
table = bq.get_table(TABLE_ID)

In [30]:
table

Table(TableReference(DatasetReference('maven-ai-advanced-course', 'google_redis_llms'), 'hotel_datasets'))

In [31]:
table.schema

[SchemaField('hotel_name', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('hotel_description', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('review_title', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('review_text', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('rate', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('tripdate', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('hotel_url', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('hotel_image', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('price_range', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('rating_value', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('review_count', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('street_address', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('locality', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('country', 'STRING', 'NULLABLE', None, None

Make sure to enable API key: https://cloud.google.com/bigquery/docs/explore-data-colab

## Load data from BigQuery - Hotel reviews.csv

In [32]:
query = f"""
SELECT * FROM `{PROJECT_ID}.google_redis_llms.{TABLE_NAME}`
"""

results = bq.query(query).to_dataframe()

In [34]:
results

,hotel_name,hotel_description,review_title,review_text,rate,tripdate,hotel_url,hotel_image,price_range,rating_value,review_count,street_address,locality,country,id
0,Citadines Tour Eiffel Paris,None,No pride of ownership,If you’ve ever stayed at a hotel which owners ...,2.0,November 2023,https://www.tripadvisor.com/Hotel_Review-g1871...,https://media-cdn.tripadvisor.com/media/photo-...,$$ (Based on Average Nightly Rates for a Stand...,4.0,471,132 boulevard de Grenelle 15th Arr.,Paris,France,5598
1,Citadines Tour Eiffel Paris,None,Location Location!,"Citadines for is located in a great place, clo...",4.0,April 2023,https://www.tripadvisor.com/Hotel_Review-g1871...,https://media-cdn.tripadvisor.com/media/photo-...,$$ (Based on Average Nightly Rates for a Stand...,4.0,471,132 boulevard de Grenelle 15th Arr.,Paris,France,5599
2,Citadines Tour Eiffel Paris,None,Amazing stay!,We absolutely loved this hotel! The staff was ...,5.0,November 2023,https://www.tripadvisor.com/Hotel_Review-g1871...,https://media-cdn.tripadvisor.com/media/photo-...,$$ (Based on Average Nightly Rates for a Stand...,4.0,471,132 boulevard de Grenelle 15th Arr.,Paris,France,5600
3,Citadines Tour Eiffel Paris,None,Best Hotel in the area,This place was exactly what I needed for my lo...,5.0,June 2023,https://www.tripadvisor.com/Hotel_Review-g1871...,https://media-cdn.tripadvisor.com/media/photo-...,$$ (Based on Average Nightly Rates for a Stand...,4.0,471,132 boulevard de Grenelle 15th Arr.,Paris,France,5601
4,Citadines Tour Eiffel Paris,None,Magnificent city.,Dissapointed . Tower Eiffel was not as i excpe...,4.0,September 2023,https://www.tripadvisor.com/Hotel_Review-g1871...,https://media-cdn.tripadvisor.com/media/photo-...,$$ (Based on Average Nightly Rates for a Stand...,4.0,471,132 boulevard de Grenelle 15th Arr.,Paris,France,5602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5992,The Gate,Situated a few minutes’ walk from London’s Ald...,One night stay. Studio apartment,Friendly & welcoming staff. Modern & well main...,5.0,September 2023,https://www.tripadvisor.com/Hotel_Review-g1863...,https://media-cdn.tripadvisor.com/media/photo-...,$$ (Based on Average Nightly Rates for a Stand...,5.0,920,41 White Church Lane,London,United Kingdom,3356
5993,The Gate,Situated a few minutes’ walk from London’s Ald...,Fantastic!,Had to book a last minute room after missing o...,5.0,August 2023,https://www.tripadvisor.com/Hotel_Review-g1863...,https://media-cdn.tripadvisor.com/media/photo-...,$$ (Based on Average Nightly Rates for a Stand...,5.0,920,41 White Church Lane,London,United Kingdom,3357
5994,The Gate,Situated a few minutes’ walk from London’s Ald...,Great stay!,"We had a great time at the Gate Hotel, the fac...",5.0,September 2023,https://www.tripadvisor.com/Hotel_Review-g1863...,https://media-cdn.tripadvisor.com/media/photo-...,$$ (Based on Average Nightly Rates for a Stand...,5.0,920,41 White Church Lane,London,United Kingdom,3358
5995,The Gate,Situated a few minutes’ walk from London’s Ald...,Heartfelt Appreciation for Mohammad's Outstand...,The Gate London Hotel provides a memorable and...,5.0,September 2023,https://www.tripadvisor.com/Hotel_Review-g1863...,https://media-cdn.tripadvisor.com/media/photo-...,$$ (Based on Average Nightly Rates for a Stand...,5.0,920,41 White Church Lane,London,United Kingdom,3359


## 3. Generate Embeddings

### Create text embeddings with Vertex AI embedding model
Use the [Vertex AI API for text embeddings](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings), developed by Google.

> Text embeddings are a dense vector representation of a piece of content such that, if two pieces of content are semantically similar, their respective embeddings are located near each other in the embedding vector space. This representation can be used to solve common NLP tasks, such as:
> - **Semantic search**: Search text ranked by semantic similarity.
> - **Recommendation**: Return items with text attributes similar to the given text.
> - **Classification**: Return the class of items whose text attributes are similar to the given text.
> - **Clustering**: Cluster items whose text attributes are similar to the given text.
> - **Outlier Detection**: Return items where text attributes are least related to the given text.

### Define embedding helper function
We define a helper function, `embedding_model_with_backoff`, to create embeddings from a list of texts while making it resilient to [Vertex AI API quotas](https://cloud.google.com/vertex-ai/docs/quotas) via [exponential backoff](https://en.wikipedia.org/wiki/Exponential_backoff).

We also define a method to convert an array of floats to a byte string for efficient storage in Redis (later on).



In [35]:
vertexai.init()

In [36]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.2.1
    Uninstalling sentence-transformers-3.2.1:
      Successfully uninstalled sentence-transformers-3.2.1


In [37]:
from typing import Generator, List, Any

from tenacity import retry, stop_after_attempt, wait_random_exponential
from vertexai.preview.language_models import TextEmbeddingModel

# Embedding model definition from VertexAI PaLM API
embedding_model = TextEmbeddingModel.from_pretrained(text_embedding)
VECTOR_DIMENSIONS = 768
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(3))
def embed_text(text=[]):
    embeddings = embedding_model.get_embeddings(text)
    return [each.values for each in embeddings]

# Convert embeddings to bytes for Redis storage
def convert_embedding(emb: List[float]):
  return np.array(emb).astype(np.float32).tobytes()

### Embed text data
At the moment, our table in BigQuery (created above), contains records of the hacker news posts that we wish to embed and make available for LLMs.

In order to conserve RAM usage of this machine, we will iterate over batches of posts from BigQuery, create embeddings, and write them to Redis, which is being used as a [vector database](https://redis.com/solutions/use-cases/vector-database).

In [38]:
import pandas as pd
import numpy as np

QUERY_TEMPLATE = f"""
SELECT id,review_title, review_text, hotel_name
FROM `{PROJECT_ID}.google_redis_llms.{TABLE_NAME}`
LIMIT {{limit}} OFFSET {{offset}};
"""

def query_bigquery_batches(
    max_rows: int,
    rows_per_batch: int,
    start_batch: int = 0
) -> Generator[pd.DataFrame, Any, None]:
    # Generate batches from a table in big query
    for offset in range(start_batch, max_rows, rows_per_batch):
        query = QUERY_TEMPLATE.format(limit=rows_per_batch, offset=offset)
        query_job = bq.query(query)
        rows = query_job.result()
        df = rows.to_dataframe()
        # Join title and text fields
        df["content"] = df.apply(lambda r: "Title: " + r.review_title + ". Content: " + r.review_text, axis=1)
        yield df


Below we define a few helper functions for processing a single row of data, writing batches to **Redis**, querying source data from **BigQuery**, and creating text embeddings with **Vertex AI**.

In [39]:
import math
from tqdm.auto import tqdm


# Redis key helper function
def redis_key(key_prefix: str, id: str) -> str:
  return f"{key_prefix}:{id}"

# Process a single dataset record
def process_record(record: dict) -> dict:
  return {
      'id': record['id'],
      'embedding': record['embedding'],
      'text': record['review_text'],
      'title': record['review_title']
  }

# Load batch of data into Redis as HASH objects
def load_redis_batch(
    redis_client: redis.Redis,
    dataset: list,
    key_prefix: str = "doc",
    id_column: str = "id",
):
    pipe = redis_client.pipeline()
    for i, record in enumerate(tqdm(dataset)):
        record = process_record(record)
        key = redis_key(key_prefix, record[id_column])
        pipe.hset(key, mapping=record)
    pipe.execute()

# Run the entire process
def create_embeddings_bigquery_redis(redis_client):
    max_rows = 1000
    rows_per_batch = 100
    bq_content_query = query_bigquery_batches(max_rows, rows_per_batch)

    for batch in tqdm(bq_content_query):
        batch_splits = np.array_split(batch, math.ceil(rows_per_batch / 5))

        all_embeddings = []

        for split_df in batch_splits:
            texts = split_df['content'].tolist()

            split_embeddings = embed_text(texts)

            all_embeddings.extend(convert_embedding(e) for e in split_embeddings)

        batch['embedding'] = all_embeddings

        records = batch.to_dict('records')
        load_redis_batch(redis_client, records)



## 4. Load Embeddings
Now that we have a function to generate BigQuery batches, create text embeddings, and write batches to Redis, we can run the single function to process our entire dataset:

Enable: https://console.cloud.google.com/apis/library/aiplatform.googleapis.com?project=maven-advanced-llm

In [40]:
create_embeddings_bigquery_redis(redis_client)

0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


  0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


  0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


  0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


RetryError: RetryError[<Future at 0x7aa7264fc0a0 state=finished raised TooManyRequests>]

In [41]:
# Validate how many records are stored in Redis
redis_client.dbsize()

300

## 5. Create Vector Index

Now that we have created embeddings that represent the text in our dataset and stored them in Redis, we will create a secondary index that enables efficient search over the embeddings. To learn more about the vector similarity features in Redis, [check out these docs](https://redis.io/docs/interact/search-and-query/search/vectors/) and [these Redis AI resources](https://github.com/RedisVentures/redis-ai-resources).

**Why do we need to enable search???**
Using Redis for vector similarity search allows us to retrieve chunks of text data that are **similar** or **relevant** to an input question or query. This will be extremely helpful for our sample generative ai / LLM application.

In [42]:
from redis.commands.search.field import (
    NumericField,
    TagField,
    TextField,
    VectorField,
)
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query


INDEX_NAME = "google:idx"
PREFIX = "doc:"
VECTOR_FIELD_NAME = "embedding"

# Store vectors in redis and create index
def create_redis_index(
    redis_client: redis.Redis,
    vector_field_name: str = VECTOR_FIELD_NAME,
    index_name: str = INDEX_NAME,
    prefix: list = [PREFIX],
    dim: int = VECTOR_DIMENSIONS
  ):

    # Construct index
    try:
        redis_client.ft(index_name).info()
        print("Existing index found. Dropping and recreating the index", flush=True)
        redis_client.ft(index_name).dropindex(delete_documents=False)
    except:
        print("Creating new index", flush=True)

    # Create new index
    redis_client.ft(index_name).create_index(
        (
            VectorField(
                vector_field_name, "FLAT",
                {
                    "TYPE": "FLOAT32",
                    "DIM": dim,
                    "DISTANCE_METRIC": "COSINE",
                }
            )
        ),
        definition=IndexDefinition(prefix=prefix, index_type=IndexType.HASH)
    )

In [43]:
# Create index
create_redis_index(redis_client)

Creating new index


In [44]:
# Inspect index attributes
redis_client.ft(INDEX_NAME).info()

{'index_name': 'google:idx',
 'index_options': [],
 'index_definition': [b'key_type',
  b'HASH',
  b'prefixes',
  [b'doc:'],
  b'default_score',
  b'1'],
 'attributes': [[b'identifier',
   b'embedding',
   b'attribute',
   b'embedding',
   b'type',
   b'VECTOR',
   b'algorithm',
   b'FLAT',
   b'data_type',
   b'FLOAT32',
   b'dim',
   768,
   b'distance_metric',
   b'COSINE']],
 'num_docs': 300,
 'max_doc_id': 300,
 'num_terms': 0,
 'num_records': 300,
 'inverted_sz_mb': '0',
 'vector_index_sz_mb': '3.021514892578125',
 'total_inverted_index_blocks': 0,
 'offset_vectors_sz_mb': '0',
 'doc_table_size_mb': '0.02117156982421875',
 'sortable_values_size_mb': '0',
 'key_table_size_mb': '0.009430885314941406',
 'tag_overhead_sz_mb': '0',
 'text_overhead_sz_mb': '0',
 'total_index_memory_sz_mb': '0.030602455139160156',
 'geoshapes_sz_mb': '0',
 'records_per_doc_avg': '1',
 'bytes_per_record_avg': '0',
 'offsets_per_term_avg': '0',
 'offset_bits_per_record_avg': 'nan',
 'hash_indexing_failure

In [45]:
# Retreive single HASH from Redis
key = redis_client.keys()[1]
redis_client.hgetall(key)

{b'title': b'Fab location , decent stay .',
 b'text': b'Daily cleaning not available , clear own garbage , breakfast is a bit overpriced , reception staff is very helpful .  We stayed for 3 nights - it was decent , supermarket next door , MC Donalds , sushi , next door , diagonally opp restaurant awesome food , very good location \xe2\x80\xa6\xe2\x80\xa6 we walked to Eiffel Tower 15/20 mins max .',
 b'id': b'5629',
 b'embedding': b'\x99\xc8\x8d<v\x98\x00\xbb\xd5?\x0c\xbd\x81\x03\x94\xbc\x1d%\xd9<\x11f\xb0<\xf9\x97\x02=7\xa8\xb1\xbc\xc0\xad\x91\xbb\xd4\x02\x90\xbc\xb0x\xb6\xbd\\\xf8i:\xf7\xbb\xa8=\xe0+b\xbc\x15\xd3L<\x12%\x9f\xbcN-\x1a<\xc8\xf5W\xbdgh\x9f\xbb\x0fC\x82;\x9dI>\xbb\x14y\x9c;.\x90\xce\xbc\xdb-#\xbd\xd7\xc0H=\x99\xbb,\xbc\x84\xb1\xbc;H\xee0=\xaai7\xbd{^\x92<\xcby@=\xf4W\x11\xbdx\xe1\r=\x86)q\xbd\x96\xb0C\xbd\xe5\x19@<\xcf\xca\x0f=<?H\xbd\x04\x10\xca<\xfek\xa4\xbc\xf0\xa8\x8d</\x8d%\xbc\x82n\x82\xbc\xef\x82\x86\xbc^l\x1d=F\xb5\x94\xbc \xd4\xd8<=\x89\xeb\xbcH:B=\xbe\t\xb0\xbd\

At this point, our **Redis** datastore is completely loaded with a subset of data from **BigQuery** including text embeddings created with **Vertex AI** PaLM APIs.

# Build LLM applications
With Redis fully loaded as a vector database and powerful PaLM APIs at our disposal, we can build a number of AI applications on this stack. Below we will briefly describe each of these applications and use cases

- **Document Retrieval** - search through documents to return only the most relevant to a given query.
- **Product Recommendations** - recommend products with similar attributes and descriptions to a product the shopper likes.
- **Chatbots** - provide a conversational interface for information retrieval or customer service.
- **Text Summarization & Generation** - Generate new copy from sources of relevant information to accelerate team output.
- **Fraud/Anomaly Detection** - identify anomalous and potentially fraudulent events, transactions, or items based on attribute similarity of other known entities.

# LLM Design Patterns

In order to build these kinds of apps, below we highlight 4 technical design patterns and techniques where Redis Enterprise comes in handy to boost LLM performance:

- **Semantic Search**
- **Retrieval Augmented Generation (RAG)**
- **Caching**
- **Memory**

Leveraging some combination of these patterns is recommended best practice, derived from enterprise use cases and open source users all over the world.

### Simple Semantic Search


**Semantic Search**, in the context of Large Language Models (LLMs), is a sophisticated search technique that goes beyond *literal* keyword matching to understand the contextual meaning and intent behind user queries. Leveraging the power of Google's Vertex AI platform and Redis' vector database capabilities, semantic search can map and extract deep-level knowledge from vast text datasets, including nuanced relationships and hidden patterns.

This allows applications to return search results that are contextually relevant, enhancing user experience by offering meaningful responses, even to complex or ambiguous search terms. Thus, semantic search not only boosts the accuracy and relevancy of search results but also empowers applications to interact with users in a more human-like, intuitive manner.

The general process of semantic search includes 3 steps:
1. Create query vector
2. Perform vector search
3. Review and return results

In [50]:
# 1. Create query vector
query = "What is the best hotel close to the Barceloneta?"
query_vector = embed_text([query])[0]

# Our query has been converted to a list of floats (this is a truncated view)
query_vector[:10]

[0.02385716140270233,
 0.011936000548303127,
 -0.07101608067750931,
 -0.03345551714301109,
 0.02387220412492752,
 -0.01795085333287716,
 -0.003341569332405925,
 0.02989981137216091,
 -0.008218161761760712,
 0.059721898287534714]

In [51]:
# Helper method to perform KNN similarity search in Redis
def similarity_search(query: str, k: int, return_fields: tuple, index_name: str = INDEX_NAME) -> list:
    # create embedding from query text
    query_vector = embed_text([query])[0]
    # create redis query object
    redis_query = (
        Query(f"*=>[KNN {k} @{VECTOR_FIELD_NAME} $embedding AS score]")
            .sort_by("score")
            .return_fields(*return_fields)
            .paging(0, k)
            .dialect(2)
    )
    # execute the search
    results = redis_client.ft(index_name).search(
        redis_query, query_params={"embedding": convert_embedding(query_vector)}
    )
    return pd.DataFrame([t.__dict__ for t in results.docs ]).drop(columns=["payload"])


In [52]:
# 2. Perform vector similarity search with given query
results = similarity_search(query, k=5, return_fields=("score", "title", "text"))

In [53]:
# 3. Review and return the results
display(results)

,id,score,title,text
0,doc:5721,0.366459310055,Great location!,This hotel is in such a great location. Our r...
1,doc:5635,0.372870266438,One of my Best Stays Ever,"What a fantastic stay, and great value for mon..."
2,doc:4852,0.393727958202,Welcoming hotel,"Good location, near to Metro and some nice res..."
3,doc:5896,0.397796750069,Eurotrip,Amazing. The staff Pedro was very kind with us...
4,doc:4882,0.400456428528,"Nice hotel, great location and great staff.",This is a nice hotel in a great location to th...


Results above indicate that our search for recommended operating systems for software devs yielded some posts from Hacker News that might be helpful in answering this question.

**Interested in tuning the search results?**
- Try using a different [Distance Metric](https://redis.io/docs/interact/search-and-query/search/vectors/#creation-attributes-per-algorithm)
- Try using a different [Index Type](https://redis.io/docs/interact/search-and-query/search/vectors/#flat)

### Retrieval Augmented Generation (RAG)

**Retrieval Augmented Generation** (RAG), within the scope of Large Language Models (LLMs), is a technique that combines the knowledge of domain-specific data and generative models to enhance the production of contextually-rich question responses. In essence, *RAG* functions by retrieving relevant information from a knowledge base of documents or data before proceeding to generate a response. This allows generalized foundation models to gain access to these datasources at runtime, and is NOT the same thing as fine-tuning.

RAG exploits the strengths of Redis as a low-latency vector database for efficient retrieval operations and Google's Vertex AI to generate a coherent text response. In LLM applications, RAG enables a deeper comprehension of context, returning highly nuanced responses, even to intricate queries. This pattern enhances the interactive capability of applications, delivering more precise and informative responses, thereby significantly enriching the user experience.


In order to build a RAG pipeline for question answering, we need to use Vertex PaLM API for text generation (`text-bison@001`).

In [54]:
from vertexai.preview.language_models import TextGenerationModel
from vertexai.preview.language_models import ChatModel, InputOutputTextPair
from vertexai.generative_models import GenerativeModel
# Define generation model
model = GenerativeModel("gemini-1.5-flash-002")

response = model.generate_content("What is a large language model?")

print("Example response:\n", response.text)


Example response:
 A large language model (LLM) is a type of artificial intelligence (AI) that's trained on a massive amount of text data.  This allows it to understand, generate, and translate human language with remarkable fluency.  Think of it as a sophisticated pattern-matching machine that has learned the statistical relationships between words and phrases.

Here's a breakdown of key aspects:

* **Massive Dataset:** LLMs are trained on datasets containing billions, even trillions, of words from various sources like books, articles, code, and websites.  The sheer scale of this data is crucial to their ability.

* **Deep Learning:**  They use deep learning techniques, specifically a type of neural network called a transformer, to process and understand the data.  Transformers excel at understanding the context and relationships between words in a sentence, even long and complex ones.

* **Predictive Text Generation:**  At their core, LLMs predict the next word in a sequence.  They d

In order to be able to answer questions **while referencing domain-specific sources** (like our sample hackernews dataset), we must build a RAG pipeline:

1. First perform **Semantic Search** with the user query on the knowledge base (stored in Redis) to find relevant sources that will help the language model answer and respond intelligently.

2. The sources (called context) are "stuffed" into the prompt (input).

3. Lastly, the full prompt is passed on to the language model for text generation.

In [55]:
def create_prompt(prompt_template: str, **kwargs) -> str:
  return prompt_template.format(**kwargs)

def rag(query: str, prompt: str, verbose: bool = True) -> str:
    """
    Simple pipeline for performing retrieval augmented generation with
    Google Vertex PaLM API and Redis Enterprise.
    """
    # Perform a vector similarity search in Redis
    if verbose:
        print("Pulling relevant data sources from Redis", flush=True)
    relevant_sources = similarity_search(query, k=3, return_fields=("text",))
    if verbose:
        print("Relevant sources found!", flush=True)
    # Combine the relevant sources and inject into the prompt
    sources_text = "-" + "\n-".join([source for source in relevant_sources.text.values])
    full_prompt = create_prompt(
        prompt_template=prompt,
        sources=sources_text,
        query=query
      )
    if verbose:
        print("\nFull prompt:\n\n", full_prompt, flush=True)
    # Perform text generation to get a response from PaLM API
    response = model.generate_content(full_prompt)
    return response.text



Below is an example prompt template. Feel free to edit and tweak the initial sentence that sets the context for the language model to perform the action we are anticipating. The process of tuning and iterating on prompt design is widely refered to as "*prompt engineering*".

In [56]:
PROMPT = """You are a helpful virtual technology and IT assistant. Use the hotel reviews below as relevant context and sources to help answer the user question. Don't blindly make things up.

SOURCES:
{sources}

QUESTION:
{query}?

ANSWER:"""



In [57]:
query = "Best hotel near the Louvre in Paris?"
response = rag(query=query, prompt=PROMPT)
print(response)

Pulling relevant data sources from Redis
Relevant sources found!

Full prompt:

 You are a helpful virtual technology and IT assistant. Use the hotel reviews below as relevant context and sources to help answer the user question. Don't blindly make things up.

SOURCES:
-Fabulous stay for our family of 3 for a 24 hour whistle-stop visit to Paris.  Hotel is incredibly well located for restaurants, shops, sightseeing and transport. Walking distance to the Louvre, along Rue Saint-Honore or the Seine with so many lovely boulangeries nearby!  Service was excellent and our 8 year old loved the afternoon cakes and treats.  We were visiting for a special birthday and we had balloons and extra treats delivered to the room which were a lovely touch.
-Perfect place if you want central location to discover Paris on foot. Easy tube access from airport and located in fantastic location. Staff is also very helpful and always ready to recommend good place to eat. Rooms have all you need and are clean, 

In [58]:
query = "What are some amazing hotels near Big ben?"
response = rag(query=query, prompt=PROMPT)
print(response)

Pulling relevant data sources from Redis
Relevant sources found!

Full prompt:

 You are a helpful virtual technology and IT assistant. Use the hotel reviews below as relevant context and sources to help answer the user question. Don't blindly make things up.

SOURCES:
-The hotel was great. Clean, free  breakfast and snacks and the staff was great. Tej from the breakfast staff was very friendly and helpful. The location was also convenient and very close to the tube station.
-This hotel was lovely and in a very good location, the rooms were clean and a reasonable size. The beds were comfy and the staff were very friendly and helpful. Very reasonable price too. Would definitely stay again.
-Positives: fantastic location, friendly and helpful staff, decent fitness suite, rooms are clean and tidy, shower pretty powerful, and the breakfast is excellent.   Negatives: only real grumble is the rooms. While they are clean and tidy, there were a few little things that weren’t as good as they co

Clearly this example dataset (hackernews) is not the only example we could work with and it 's certainly not "production" ready out of the gate. This is also only utilizing a subset (1000 records) of the actual data for teaching purposes.

However, this example demonstrates how you can combine external sources of data and LLMs to surface more useful information.

### LLM Caching

**LLM Caching** is an advanced strategy used to optimize the performance of Large Language Model (LLM) applications. Utilizing the ultra-fast, in-memory data store of Redis, LLM Caching enables the storage and quick retrieval of pre-computed responses generated by Google's Vertex AI (PaLM). This means the computationally expensive process of response generation, especially for repetitive queries, is significantly reduced, resulting in faster response times and efficient resource utilization. This pairing of Google's powerful generative AI capabilities with Redis' high-performance caching system thus facilitates a more scalable and performant architecture for LLM applications, improving overall user experience and application reliability.

There are primarily two modes of caching for LLMs:
- Standard Caching
- Semantic Caching

#### Standard Caching

Standard caching for LLMs involves simply matching an exact phrase or prompt that has been provided before. We can return the previously used response from the LLM in order to speed up the throughput of the system overall and reduce redundant computation.

In [59]:
# Some boiler plate helper methods
import hashlib

def hash_input(prefix: str, _input: str):
    return prefix + hashlib.sha256(_input.encode("utf-8")).hexdigest()

def standard_check(key: str):
  # function to perform a standard cache check
    res = redis_client.hgetall(key)
    if res:
      return res[b'response'].decode('utf-8')

def cache_response(query: str, response: str):
    key = hash_input("llmcache:", query)
    redis_client.hset(key, mapping={"prompt": query, "response": response})

# LLM Cache wrapper / decorator function
def standard_llmcache(llm_callable):
    def wrapper(*args, **kwargs):
        # Check LLM Cache first
        key = hash_input("llmcache:", *args, **kwargs)
        response = standard_check(key)
        # Check if we have a cached response we can use
        if response:
            return response
        # Otherwise execute the llm callable here
        response = llm_callable(*args, **kwargs)
        cache_response(query, response)
        return response

    return wrapper

In [60]:
# Define a function that invokes the PaLM API wrapped with a cache check

@standard_llmcache
def ask_palm(query: str):
  prompt = PROMPT
  response = rag(query, prompt, verbose=False)
  return response

In [61]:
%%time

query = "What are some amazing hotels near Big ben?"

ask_palm(query)

CPU times: user 23.4 ms, sys: 4.19 ms, total: 27.6 ms
Wall time: 1.14 s


"The provided reviews don't name the hotel,  nor do they state its proximity to Big Ben.  Therefore, I cannot answer your question using only the given information.  The reviews describe hotels that are conveniently located near a tube station and have positive attributes like clean rooms, friendly staff, and good breakfasts, but lack the crucial information about their location relative to Big Ben.\n"

Now if we ask the same question again -- we should get the same response in near real-time.

In [62]:
%%time

ask_palm(query)

CPU times: user 1.14 ms, sys: 11 µs, total: 1.15 ms
Wall time: 1.19 ms


"The provided reviews don't name the hotel,  nor do they state its proximity to Big Ben.  Therefore, I cannot answer your question using only the given information.  The reviews describe hotels that are conveniently located near a tube station and have positive attributes like clean rooms, friendly staff, and good breakfasts, but lack the crucial information about their location relative to Big Ben.\n"

#### Semantic Caching - Assignment
Implement Semantic Caching and try to write data to GCP big query and retrieve from it - you can also use json locally


### Memory

Giving your application access to "memory" for chat history is a common technique to improve the models ability to reason through recent or past conversations, gain context from previous answers, and thus provide a more accurate and acceptable response.

Below we setup simple helper functions to persist and load conversation history in a Redis List data structure.

In [63]:
import json

def add_message(prompt: str, response: str):
    msg = {
        "prompt": prompt,
        "response": response
    }
    redis_client.lpush("chat-history", json.dumps(msg))

def get_messages(k: int = 5):
    return [json.loads(msg) for msg in redis_client.lrange("chat-history", 0, k)]

In [64]:

query = "Do you have any advice for getting started in the tech field as a software dev?"
response = rag(query, PROMPT, verbose=False)

print(response)

add_message(query, response)

The provided text focuses on reviews of a hostel and a mojito class, and doesn't offer any information relevant to starting a career in software development.  Therefore, I cannot provide advice on that topic based on these sources.



In [65]:
query = "What if I am still in college, any tips there?"
response = rag(query, PROMPT, verbose=False)

print(response)

add_message(query, response)

Based on the reviews, St Christopher's Hostel seems geared towards socializing.  If you're a college student, this could be a great option for meeting people. The reviews mention free nightly social events, making it easy to connect with fellow travelers.  However, keep in mind that there are no in-room kitchens, so plan accordingly.  Also, be mindful of the entry/exit rules regarding drinks, as mentioned in one review.



In [66]:
get_messages()

[{'prompt': 'What if I am still in college, any tips there?',
  'response': "Based on the reviews, St Christopher's Hostel seems geared towards socializing.  If you're a college student, this could be a great option for meeting people. The reviews mention free nightly social events, making it easy to connect with fellow travelers.  However, keep in mind that there are no in-room kitchens, so plan accordingly.  Also, be mindful of the entry/exit rules regarding drinks, as mentioned in one review.\n"},
 {'prompt': 'Do you have any advice for getting started in the tech field as a software dev?',
  'response': "The provided text focuses on reviews of a hostel and a mojito class, and doesn't offer any information relevant to starting a career in software development.  Therefore, I cannot provide advice on that topic based on these sources.\n"}]

In [69]:
query = "What if I want to have a great breakfast?"
response = rag(query, PROMPT, verbose=False)

print(response)

add_message(query, response)


Based on guest reviews, this hotel offers a fantastic breakfast with a wide variety of options.  Reviewers specifically mention pancakes, waffles, frozen yogurt, fresh juices (with options to mix your own), salmon, and bio foods.  The breakfast service also receives high praise, with several guests mentioning excellent service from staff members like Pal and his colleagues from the Philippines, and Fabrice, the manager.  If you want a great breakfast experience, this hotel is likely a good choice.



# Clean up

In [70]:
# Clean up bigquery
bq.delete_table(TABLE_ID, not_found_ok=True)

bq.delete_dataset(
    DATASET_ID, delete_contents=True, not_found_ok=True
)


In [71]:
# Clean up redis
!redis-stack-server stop

Starting redis-stack-server, database path /var/lib/redis-stack
9167:C 12 Dec 2024 21:21:47.159 * oO0OoO0OoO0Oo Redis is starting oO0OoO0OoO0Oo
9167:C 12 Dec 2024 21:21:47.159 * Redis version=7.4.1, bits=64, commit=00000000, modified=0, pid=9167, just started
9167:C 12 Dec 2024 21:21:47.159 * Configuration loaded
9167:M 12 Dec 2024 21:21:47.159 * monotonic clock: POSIX clock_gettime
                _._                                                  
           _.-``__ ''-._                                             
      _.-``    `.  `_.  ''-._           Redis Community Edition      
  .-`` .-```.  ```\/    _.,_ ''-._     7.4.1 (00000000/0) 64 bit
 (    '      ,       .-`  | `,    )     Running in standalone mode
 |`-._`-...-` __...-.``-._|'` _.-'|     Port: 6379
 |    `-._   `._    /     _.-'    |     PID: 9167
  `-._    `-._  `-./  _.-'    _.-'                                   
 |`-._`-._    `-.__.-'    _.-'_.-'|                                  
 |    `-._`-._        _.-'_.-' 